In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pwd

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/kaggle/working


In [3]:
!cp -r /kaggle/input/plantdisease/PlantVillage/ /kaggle/working/PlantVillage/

In [4]:
import os
import shutil

In [5]:
for i,j,k in os.walk('/kaggle/working/PlantVillage/'):
    print(i)
    if 'Potato' in i:
        shutil.rmtree(i)
for i,j,k in os.walk('/kaggle/working/PlantVillage/'):
    print(i)
    if 'Pepper' in i:
        shutil.rmtree(i)
    

/kaggle/working/PlantVillage/
/kaggle/working/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus
/kaggle/working/PlantVillage/Tomato_Leaf_Mold
/kaggle/working/PlantVillage/Tomato_Late_blight
/kaggle/working/PlantVillage/Tomato_healthy
/kaggle/working/PlantVillage/PlantVillage
/kaggle/working/PlantVillage/PlantVillage/Pepper__bell___healthy
/kaggle/working/PlantVillage/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus
/kaggle/working/PlantVillage/PlantVillage/Tomato_Leaf_Mold
/kaggle/working/PlantVillage/PlantVillage/Tomato_Late_blight
/kaggle/working/PlantVillage/PlantVillage/Tomato_healthy
/kaggle/working/PlantVillage/PlantVillage/Pepper__bell___Bacterial_spot
/kaggle/working/PlantVillage/PlantVillage/Potato___Late_blight
/kaggle/working/PlantVillage/PlantVillage/Tomato_Septoria_leaf_spot
/kaggle/working/PlantVillage/PlantVillage/Potato___Early_blight
/kaggle/working/PlantVillage/PlantVillage/Tomato_Bacterial_spot
/kaggle/working/PlantVillage/PlantVillage/Potato___healthy
/kaggle/wor

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load the dataset from the directory
dataset_path = "/kaggle/working/PlantVillage/"
batch_size = 32
img_height = 224  # Adjust the height as needed
img_width = 224   # Adjust the width as needed

# Load training dataset
train_dataset = image_dataset_from_directory(
    dataset_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',  # 'int' for integer labels or 'categorical' for one-hot encoding
    shuffle=True,      # Shuffle the dataset
    seed=123,          # Set a seed for reproducibility
    validation_split=0.2,  # Use 20% of the data for validation
    subset='training'  # 'training' subset
)

# Load validation dataset
validation_dataset = image_dataset_from_directory(
    dataset_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',  # Same as above
    shuffle=True,
    seed=123,
    validation_split=0.2,  # Same as above
    subset='validation'  # 'validation' subset
)

Found 16011 files belonging to 10 classes.
Using 12809 files for training.
Found 16011 files belonging to 10 classes.
Using 3202 files for validation.


In [11]:
num_samples = int(0.1 * 400)
dataset = train_dataset.shuffle(1000).take(num_samples)

In [26]:
class_names = len(train_dataset.class_names)

In [27]:
hw = 224
wh = 224

In [28]:
img_height, img_width = hw, wh

In [15]:
# Load ResNet50 as the base model with pre-trained weights from ImageNet
base_model = tf.keras.applications.ResNet50(input_shape=(img_height, img_width, 3),
                                            include_top=False,  # Remove the final classification layer
                                            weights='imagenet')

# Freeze the base model to prevent training its weights
base_model.trainable = False

# Define a new model with ResNet50 as the base
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),  # Reduce the tensor size
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(class_names, activation='softmax')  # Output layer
])

# Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
# model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [16]:
from tensorflow.keras.optimizers import AdamW

In [17]:
learning_rate = 1e-3
weight_decay = 1e-5  # Set weight decay rate

# Compile the model with AdamW optimizer
model.compile(optimizer=AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(dataset, validation_data=validation_dataset, epochs=5,verbose=1)


Epoch 1/5


I0000 00:00:1731441263.915994     107 service.cc:145] XLA service 0x7c7374001c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731441263.916061     107 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


 3/40 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.2135 - loss: 2.8513  

I0000 00:00:1731441270.630822     107 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 34s 296ms/step - accuracy: 0.4082 - loss: 1.9797 - val_accuracy: 0.7748 - val_loss: 0.7734
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.6709 - loss: 0.9421 - val_accuracy: 0.8295 - val_loss: 0.5566
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.7551 - loss: 0.7387 - val_accuracy: 0.8557 - val_loss: 0.4525
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 174ms/step - accuracy: 0.7977 - loss: 0.6378 - val_accuracy: 0.8663 - val_loss: 0.4118
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 258ms/step - accuracy: 0.8366 - loss: 0.5335 - val_accuracy: 0.8360 - val_loss: 0.4674


In [18]:
!rm -rf /kaggle/working/PlantVillage/PlantVillage/

In [30]:
for i in range(1,11):
    num_samples = int(i/10 * 400)
    dataset = train_dataset.shuffle(1000).take(num_samples)
    # Load ResNet50 as the base model with pre-trained weights from ImageNet
    base_model = tf.keras.applications.ResNet50(input_shape=(img_height, img_width, 3),
                                                include_top=False,  # Remove the final classification layer
                                                weights='imagenet')

    # Freeze the base model to prevent training its weights
    base_model.trainable = False

    # Define a new model with ResNet50 as the base
    model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),  # Reduce the tensor size
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Dropout for regularization
        tf.keras.layers.Dense(class_names, activation='softmax')  # Output layer
    ])

    
    
    
    learning_rate = 1e-3
    weight_decay = 1e-5  # Set weight decay rate

    # Compile the model with AdamW optimizer
    model.compile(optimizer=AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(dataset, validation_data=validation_dataset, epochs=5,verbose=1)

        
    learning_rate = 4e-5
    weight_decay = 1e-5  # Set weight decay rate

    # Compile the model with AdamW optimizer
    model.compile(optimizer=AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(dataset, validation_data=validation_dataset, epochs=5,verbose=1)
    
    
    
    import pandas as pd
    import tensorflow as tf

    # Create a list to store the labels and predictions
    labels = []
    predictions = []
    # O = []
    aou = []
    # Iterate over the validation dataset
    for batch in validation_dataset:
        images, batch_labels = batch
        batch_predictions_ = model.predict(images)
        # O.append(batch_predictions)
        batch_predictions = tf.argmax(batch_predictions_, axis=1)

        # Append the labels and predictions to the lists
        aou.extend(batch_predictions_.max(1))
        labels.extend(batch_labels.numpy())
        predictions.extend(batch_predictions.numpy())

    # Create a DataFrame to store the labels and predictions
    df = pd.DataFrame({
        'label': labels,
        'prediction': predictions,
        'aou': aou
    })
    # f = np.array(O)
    # f.reshape(-1,10)
    # Save the DataFrame to a CSV file
    df.to_csv(f'/kaggle/working/res/validation_results_{i}.csv', index=False)

    # Save the predictions to a dataset
    # prediction_dataset = tf.data.Dataset.from_tensor_slices(predictions)


Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 30s 261ms/step - accuracy: 0.3750 - loss: 1.9406 - val_accuracy: 0.7936 - val_loss: 0.6778
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.6937 - loss: 0.8836 - val_accuracy: 0.8601 - val_loss: 0.4694
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 151ms/step - accuracy: 0.7914 - loss: 0.6467 - val_accuracy: 0.8573 - val_loss: 0.4539
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 153ms/step - accuracy: 0.8287 - loss: 0.5243 - val_accuracy: 0.8716 - val_loss: 0.3863
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.7895 - loss: 0.5542 - val_accuracy: 0.8776 - val_loss: 0.3684
Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 26s 262ms/step - accuracy: 0.8430 - loss: 0.4467 - val_accuracy: 0.9044 - val_loss: 0.3066
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 151ms/step - accuracy: 0.8514 - loss: 0.4341 - val_accuracy: 0.9054 - val_loss: 0.2993
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 230ms/step - accuracy: 0.8811 - loss: 0.3520 - val_accuracy: 0.

In [23]:
os.makedirs('/kaggle/working/res/',exist_ok=True)

In [32]:
!zip -r zi.zip '/kaggle/working/res/' 

  adding: kaggle/working/res/ (stored 0%)
  adding: kaggle/working/res/validation_results_8.csv (deflated 70%)
  adding: kaggle/working/res/validation_results_9.csv (deflated 70%)
  adding: kaggle/working/res/validation_results_6.csv (deflated 69%)
  adding: kaggle/working/res/validation_results_5.csv (deflated 69%)
  adding: kaggle/working/res/validation_results_1.csv (deflated 65%)
  adding: kaggle/working/res/validation_results_3.csv (deflated 67%)
  adding: kaggle/working/res/validation_results_2.csv (deflated 66%)
  adding: kaggle/working/res/validation_results_7.csv (deflated 69%)
  adding: kaggle/working/res/validation_results_10.csv (deflated 71%)
  adding: kaggle/working/res/validation_results_4.csv (deflated 68%)
